In [1]:
import os
import shutil

import teotil_report as rep
from docx import Document
from docx.shared import Pt

# Generate TEOTIL2 Word report

Inserts maps and tables into Word using the structure required for the annual TEOTIL report. Text can then be added/edited afterwards.

In [2]:
# User input for output file. A new file will be created named
# "../report_{year}/teotil_report_{year}_draft{draft_no}.docx"
# NOTE: If this already exists, it will be overwritten!
year = 2021
draft_no = 1

# Whether to use 'legacy' data or 'TEOTIL2' data for early years.
# NIVA's database is not complete for early years, and it seems John
# Rune did some manual patching to fill gaps (using MS Access). Setting
# 'patch_legacy' = False will use model results from GitHub from TEOTIL2
# for all years. These results are "reproducible", but contain gaps for
# some early years. Use 'patch_legacy' = True to use the data compiled
# by Jose based on John Rune's Access databases. This is more complete,
# but the methodology and data sources for the patching are unknown.
# When 'patch_legacy' = True, legacy data will be used for years
# <= 'cutoff_year'.
patch_legacy = True
cutoff_year = 1995

data_fold = f"../report_{year}/data"
legacy_data_fold = f"../report_2020/jose_data_updated_2018-20"

In [3]:
# Dict mapping headings in the Word template to tables/images that should be inserted below them
# If the value is None, the file path can be constructed from the heading alone
data_dict = {
    "Kart over kildepunkter": [
        f"../report_{year}/plots/sewage.png",
        f"../report_{year}/plots/industry.png",
        f"../report_{year}/plots/aquaculture.png",
    ],
    "Norges kystområder: fosfor (tonn)": None,
    "Norges kystområder: nitrogen (tonn)": None,
    "Sverige – Strømtangen fyr: fosfor (tonn)": None,
    "Sverige – Strømtangen fyr: nitrogen (tonn)": None,
    "Indre Oslofjord: fosfor (tonn)": None,
    "Indre Oslofjord: nitrogen (tonn)": None,
    "Svenskegrensa – Lindesnes: fosfor (tonn)": None,
    "Svenskegrensa – Lindesnes: nitrogen (tonn)": None,
    "Lindesnes – Stad: fosfor (tonn)": None,
    "Lindesnes – Stad: nitrogen (tonn)": None,
    "Stad – Russland: fosfor (tonn)": None,
    "Stad – Russland: nitrogen (tonn)": None,
    "Glomma: fosfor (tonn)": None,
    "Glomma: nitrogen (tonn)": None,
    "Vest-Viken: fosfor (tonn)": None,
    "Vest-Viken: nitrogen (tonn)": None,
    "Agder: fosfor (tonn)": None,
    "Agder: nitrogen (tonn)": None,
    "Rogaland: fosfor (tonn)": None,
    "Rogaland: nitrogen (tonn)": None,
    "Hordaland: fosfor (tonn)": None,
    "Hordaland: nitrogen (tonn)": None,
    "Sogn og Fjordane: fosfor (tonn)": None,
    "Sogn og Fjordane: nitrogen (tonn)": None,
    "Møre og Romsdal: fosfor (tonn)": None,
    "Møre og Romsdal: nitrogen (tonn)": None,
    "Trøndelag: fosfor (tonn)": None,
    "Trøndelag: nitrogen (tonn)": None,
    "Nordland: fosfor (tonn)": None,
    "Nordland: nitrogen (tonn)": None,
    "Troms: fosfor (tonn)": None,
    "Troms: nitrogen (tonn)": None,
    "Finnmark: fosfor (tonn)": None,
    "Finnmark: nitrogen (tonn)": None,
    "Kartpresentajon av tilførsler av næringssalter til sjø fra vannregionene": [
        f"../report_{year}/plots/vannreg_p.png",
        f"../report_{year}/plots/vannreg_n.png",
    ],
    "Nordsjøen: fosfor (tonn)": None,
    "Nordsjøen: nitrogen (tonn)": None,
    "Norskehavet: fosfor (tonn)": None,
    "Norskehavet: nitrogen (tonn)": None,
    "Barentshavet: fosfor (tonn)": None,
    "Barentshavet: nitrogen (tonn)": None,
    "Kartpresentasjon av næringssalttilførsler til forvaltningsplanområder": [
        f"../report_{year}/plots/forvaltom_p.png",
        f"../report_{year}/plots/forvaltom_n.png",
    ],
}

In [4]:
# Copy template. NOTE: This will overwrite 'docx_path' if it already exists!
docx_path = f"../report_{year}/teotil_report_{year}_draft{draft_no}.docx"
template_path = r"../blank_template/teotil_report_blank_template.docx"
shutil.copyfile(template_path, docx_path)

# Open the document
doc = Document(docx_path)

# Set styles for 'Normal' template in this doc
style = doc.styles["Normal"]
font = style.font
font.name = "Calibri"
font.size = Pt(11)
p_format = style.paragraph_format
p_format.space_before = Pt(0)
p_format.space_after = Pt(0)

# Loop over headings in template
for para in doc.paragraphs:
    heading = para.text
    if heading in data_dict.keys():
        print(f"Processing: {heading}")

        if data_dict[heading]:
            # Processing images
            # Need to process files in reverse order as each element will be
            # inserted immediately under the header text i.e. start with last first
            flist = reversed(data_dict[heading])
            for fpath in flist:
                new_para = rep.insert_para_after(para)
                rep.insert_image(new_para, fpath)
        else:
            # Processing CSV
            fname = rep.filename_from_heading(heading)
            fpath = os.path.join(data_fold, fname)
            df = rep.read_data_table(fpath, heading)
            if patch_legacy:
                df = rep.patch_legacy_data(
                    df, legacy_data_fold, fname, heading, cutoff_year
                )
            plot_path = rep.make_plot(df, heading, fpath, year)
            new_para = rep.insert_para_after(para)
            rep.insert_image(new_para, plot_path)
            rep.insert_table(doc, para, df)

doc.save(docx_path)

print("Done.")

Processing: Kart over kildepunkter
Processing: Norges kystområder: fosfor (tonn)
Processing: Norges kystområder: nitrogen (tonn)
Processing: Sverige – Strømtangen fyr: fosfor (tonn)
Processing: Sverige – Strømtangen fyr: nitrogen (tonn)
Processing: Indre Oslofjord: fosfor (tonn)
Processing: Indre Oslofjord: nitrogen (tonn)
Processing: Svenskegrensa – Lindesnes: fosfor (tonn)
Processing: Svenskegrensa – Lindesnes: nitrogen (tonn)
Processing: Lindesnes – Stad: fosfor (tonn)
Processing: Lindesnes – Stad: nitrogen (tonn)
Processing: Stad – Russland: fosfor (tonn)
Processing: Stad – Russland: nitrogen (tonn)
Processing: Glomma: fosfor (tonn)
Processing: Glomma: nitrogen (tonn)
Processing: Vest-Viken: fosfor (tonn)
Processing: Vest-Viken: nitrogen (tonn)
Processing: Agder: fosfor (tonn)
Processing: Agder: nitrogen (tonn)
Processing: Rogaland: fosfor (tonn)
Processing: Rogaland: nitrogen (tonn)
Processing: Hordaland: fosfor (tonn)
Processing: Hordaland: nitrogen (tonn)
Processing: Sogn og Fjo